In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Federated /FL-project

/content/drive/MyDrive/Federated /FL-project


In [3]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from main import *
import argparse



In [5]:

# Class used to simplify the assignment of the arguments.
class Args():
  
  def __init__(self, dataset, model, bs, hnm, num_epochs):
    '''
    Class used to pass the-high level parameters
      -) dataset   : the dataset we are considering;
      -) model     : the model we are using;
      -) bs        : the batch size;
      -) num_epochs: number of epochs for which we will train the model.
    '''
    self.dataset = dataset    
    self.model = model      
    self.bs = bs             
    self.hnm = hnm
    self.num_epochs = num_epochs

args = Args(dataset = 'iddaCB', model = 'deeplabv3_mobilenetv2', 
            bs = 16, hnm = False, num_epochs = 3)



In [6]:
# We get the dataset and we create an instance of the model.
model = model_init(args)
train_datasets, test_datasets = get_datasets(args)
train_clients, test_clients = gen_clients(args, train_datasets, test_datasets, model)
metrics = set_metrics(args) #dict (['eval_train', 'test_same_dom', 'test_diff_dom'])


In [7]:
# We pass to our centralized environment the optimization algorithm and the 
# rate decay technique. Finally, we train our model. Thanks to WandB, we can
# keep track of the different performance.
opt_params = {'optimizer': {
                            'name'    : 'Adam',
                            'settings': {'lr'   : np.random.choice(np.linspace(0.001, 0.1, 10000))}
                            },
              'scheduler': {
                            'name'    : 'ConstantLR',
                            'settings': {'factor': 0.33}
                            }
              }


train_clients[0].set_opt(opt_params)
# Set how many steps we want to wait before plotting the loss.
#centralized.train(n_steps = 5)
# Set how many steps we want to wait before plotting the vertical lines.
#centralized.print_learning(step = 10)


server = Server(args,train_clients, test_clients,model,metrics)


In [8]:
server.train_round(train_clients)

Params freezed
epoch 1


Error: ignored

In [ ]:

# We import the metrics used to compute the goodness of our model and we test
# on our data.
from utils.stream_metrics import StreamSegMetrics
metric = StreamSegMetrics(16, 'centralizzato')
centralized.test(metric)


In [ ]:

# Here we can choose what kind of metric to show. Possible choices are:
#   -) Total samples    -) Mean IoU
#   -) Overall Acc      -) Class IoU
#   -) Mean Acc         -) Class Acc
#   -) Mean Precision   -) Class Prec
#   -) FreqW Acc 

metric.get_results()['Mean IoU']


In [ ]:

from torch.utils.data import DataLoader
train_loader = DataLoader(ds, batch_size = 2, shuffle = False, drop_last = True)


In [ ]:

centralized.model.eval()
with torch.no_grad():
  for i, (images, labels) in enumerate(train_loader):
    images = images.cuda() 
    labels = labels.cuda()
    outputs = centralized.model(images)['out']
    break


In [ ]:
outputs = centralized.model(images[0].view(1, 3, 512, 928))['out']

In [ ]:

import matplotlib.pyplot as plt
def unNormalize(tensorImage, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
        for t, m, s in zip(tensorImage, mean, std):
            t.mul_(s).add_(m)
        return tensorImage
plt.imshow(unNormalize(images[0].cpu()).permute(1,2,0))


In [ ]:

singoloOutputLogits = outputs[0]
prediction = singoloOutputLogits.argmax(0)


In [ ]:

import matplotlib.pyplot as plt
plt.imshow(unNormalize(images[0].cpu()).permute(1,2,0))
plt.imshow(prediction.cpu().numpy(), alpha = 0.4)


In [ ]:

import matplotlib as mpl

plt.imshow(labels[0].cpu().numpy(), cmap = mpl.colormaps['inferno'], vmin = 0, vmax = 16)
